In [1]:
import pandas as pd
df = pd.read_csv("29Mai2023.csv")

df.drop('Instrument', axis=1, inplace=True)
df.drop('OpenInterest', axis=1, inplace=True)
df.drop('TradeTimestamp', axis=1, inplace=True)
df.drop('Timestamp', axis=1, inplace=True)

for i in range(50):
    #df[f'High{i+1}'] = df['High'].shift(i+1) - df['High']
    #df[f'Low{i+1}'] = df['Low'].shift(i+1) - df['Low']
    df[f'Actual{i+1}'] = df['Actual'].shift(i+1) -  df['Actual']
    df[f'Volume{i+1}'] = df['Volume'].shift(i+1) - df['Volume']
    #df[f'UsdVolume{i+1}'] = df['UsdVolume'].shift(i+1) - df['UsdVolume']
    df[f'Change{i+1}'] = df['Change'].shift(i+1) - df['Change']
    df[f'BestBid{i+1}'] = df['BestBid'].shift(i+1) - df['BestBid']
    df[f'BestBidSize{i+1}'] = df['BestBidSize'].shift(i+1) - df['BestBidSize']
    df[f'BestAsk{i+1}'] = df['BestAsk'].shift(i+1) - df['BestAsk']
    df[f'BestAskSize{i+1}'] = df['BestAskSize'].shift(i+1) - df['BestAskSize']

df[f'ActualF50'] = df['Actual'].shift(-10) -  df['Actual']
df['ActualF50'] = df['ActualF50'].apply(lambda x: 1 if x >= 1 else -1 if x <= -1 else 0)
df.drop('High', axis=1, inplace=True)
df.drop('Low', axis=1, inplace=True)
df.drop('Actual', axis=1, inplace=True)
df.drop('Volume', axis=1, inplace=True)
df.drop('UsdVolume', axis=1, inplace=True)
df.drop('Change', axis=1, inplace=True)
df.drop('BestBid', axis=1, inplace=True)
df.drop('BestBidSize', axis=1, inplace=True)
df.drop('BestAsk', axis=1, inplace=True)
df.drop('BestAskSize', axis=1, inplace=True)

df = df[50:4879]
df = df.copy()

/tmp/ipykernel_20187/948566285.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Volume{i+1}'] = df['Volume'].shift(i+1) - df['Volume']
/tmp/ipykernel_20187/948566285.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Change{i+1}'] = df['Change'].shift(i+1) - df['Change']
/tmp/ipykernel_20187/948566285.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) 

In [2]:
print(df)

      Actual1  Volume1  Change1  BestBid1  BestBidSize1  BestAsk1   
50       0.47  -1.0786   0.0003      0.22       -2.0701      0.04  \
51      -0.26  -0.0600  -0.0002      0.19        1.9000     -0.04   
52       0.05  -0.0600   0.0000     -0.01        0.1998      0.39   
53       0.23  -0.0100   0.0002      0.11       -0.1997      0.00   
54      -0.14  -0.0155  -0.0001      0.03        0.0000      0.00   
...       ...      ...      ...       ...           ...       ...   
4874     0.24  -0.0400   0.0001      0.26       -7.4921      0.28   
4875     0.30  -0.3600   0.0002      0.22        7.3410      0.09   
4876    -0.01   0.0710   0.0002      0.00        0.0000      0.17   
4877     0.22  -0.0167   0.0001      0.00        0.0067      0.10   
4878     0.70  -2.8265   0.0004      0.62       -4.3477      0.64   

      BestAskSize1  Actual2  Volume2  Change2  ...  BestAsk49  BestAskSize49   
50          0.0000     0.48  -1.1386   0.0003  ...      -0.04         0.0000  \
51         

In [3]:
zeros = df[df['ActualF50'] == 0]
ones  = df[df['ActualF50'] == 1]
negatives = df[df['ActualF50'] == -1]
print(zeros['ActualF50'])
print(ones['ActualF50'])
print(negatives['ActualF50'])

50      0
51      0
52      0
53      0
54      0
       ..
4871    0
4875    0
4876    0
4877    0
4878    0
Name: ActualF50, Length: 3260, dtype: int64
87      1
90      1
91      1
92      1
113     1
       ..
4861    1
4862    1
4863    1
4864    1
4866    1
Name: ActualF50, Length: 757, dtype: int64
55     -1
56     -1
57     -1
58     -1
59     -1
       ..
4766   -1
4767   -1
4872   -1
4873   -1
4874   -1
Name: ActualF50, Length: 812, dtype: int64


In [4]:
zeros = zeros.sample(frac=1).reset_index(drop=True)
zeros = zeros[:757]
zeros = ones.sample(frac=1).reset_index(drop=True)
zeros = ones[:757]
zeros = ones.sample(frac=1).reset_index(drop=True)
zeros = ones[:757]

df = pd.concat([zeros, ones, negatives])
df = df.sample(frac=1).reset_index(drop=True)
train_dataset = df.head(1800)
test_dataset = df.tail(df.shape[0] - 1800)

In [5]:
import os
import pandas as pd
from torch.utils.data import Dataset

class CustomPriceDataset(Dataset):
    def __init__(self, dataframe, labels, transform=None, target_transform=None):
        self.dataframe = dataframe
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = list(self.dataframe.iloc[idx])
        label = self.labels.iloc[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda

trnlbl = train_dataset['ActualF50'] + 1
trnset = train_dataset.drop('ActualF50', axis=1)
trainPrice = CustomPriceDataset(trnset, trnlbl, torch.FloatTensor, Lambda(lambda y: torch.zeros(3, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)))
train_dataloader = DataLoader(trainPrice, batch_size=64)

tstlbl = test_dataset['ActualF50'] + 1
tstset = test_dataset.drop('ActualF50', axis=1)
testPrice = CustomPriceDataset(tstset, tstlbl, torch.FloatTensor)
test_dataloader = DataLoader(testPrice, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(350, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
learning_rate = 1e-2
passed = 0

In [7]:
epochs = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=1e-2, dampening=1e-3)

In [14]:
print("Start")
for t in range(epochs):
    print(f"Epoch {t+1 + passed}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
passed = passed + epochs
print("Done!")

Start
Epoch 61
-------------------------------
loss: 0.269324  [   64/ 1800]
loss: 0.185547  [ 1344/ 1800]
Test Error: 
 Accuracy: 82.5%, Avg loss: 0.432494 

Epoch 62
-------------------------------
loss: 0.126073  [   64/ 1800]
loss: 0.392242  [ 1344/ 1800]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.415473 

Epoch 63
-------------------------------
loss: 0.112207  [   64/ 1800]
loss: 0.228594  [ 1344/ 1800]
Test Error: 
 Accuracy: 82.1%, Avg loss: 0.476290 

Epoch 64
-------------------------------
loss: 0.100886  [   64/ 1800]
loss: 0.232688  [ 1344/ 1800]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.575884 

Epoch 65
-------------------------------
loss: 0.290004  [   64/ 1800]
loss: 0.140180  [ 1344/ 1800]
Test Error: 
 Accuracy: 84.2%, Avg loss: 0.407505 

Epoch 66
-------------------------------
loss: 0.101426  [   64/ 1800]
loss: 0.135936  [ 1344/ 1800]
Test Error: 
 Accuracy: 81.9%, Avg loss: 0.575266 

Epoch 67
-------------------------------
loss: 0.112296  [   64/ 1800]
loss:

In [9]:
#print(train_dataset)

In [10]:
#print(test_dataset)

In [15]:
torch.save(model, 'model5.pth')